In [ ]:
!unzip v_data.zip

In [ ]:
from pathlib import Path
import glob
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import load_img

In [ ]:
train_data_dir = Path('v_data/test/planes')
images = train_data_dir.glob("*.jpg")
train_data = []
for image in images:
  img=load_img(image, target_size=(227, 227))
  img = np.array(img)
  img = img / 255.0
  img = img.reshape(1,227,227,3)
  train_data.append([img,1])


train_data_dir = Path('v_data/test/cars')
images = train_data_dir.glob("*.jpg")
for image in images:
  img=load_img(image, target_size=(227, 227))
  img = np.array(img)
  img = img / 255.0
  img = img.reshape(1,227,227,3)
  train_data.append([img,0])
test_data = pd.DataFrame(train_data,columns=['image','label'],index = None)
test_data

,image,label
0,"[[[[0.47058824 0.59607843 0.65490196], [0.4666...",1
1,"[[[[0.58431373 0.6627451 0.76862745], [0.5843...",1
2,"[[[[0.46666667 0.63921569 0.81568627], [0.4666...",1
3,"[[[[0.28235294 0.30980392 0.34117647], [0.5568...",1
4,"[[[[0.41176471 0.44313725 0.4 ], [0.3098...",1
...,...,...
95,"[[[[0.69019608 0.60784314 0.15686275], [0.6901...",0
96,"[[[[0.62745098 0.65098039 0.70588235], [0.6274...",0
97,"[[[[0.29019608 0.3372549 0.4 ], [0.4745...",0
98,"[[[[0.91764706 0.91764706 0.90980392], [0.9137...",0


In [ ]:
train_data_dir = 'v_data/train'
validation_data_dir = 'v_data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16
img_width, img_height = 224, 224

In [ ]:
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

model.fit_generator(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)

FileNotFoundError: ignored

In [ ]:
model_ann = Sequential()
model_ann.add(Dense(16, input_shape=input_shape, activation='relu'))
model_ann.add(Dropout(0.4))
model_ann.add(Dense(32, activation='relu'))
model_ann.add(Dropout(0.6))
model_ann.add(Flatten())
model_ann.add(Dense(64,input_shape=input_shape, activation='softmax'))
model_ann.add(Dropout(0.2))
model_ann.add(Dense(1))
model_ann.add(Activation('sigmoid'))
model_ann.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:

model_ann.fit_generator(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.summary()

In [ ]:
model_ann.summary()

In [ ]:
import numpy as np

image = load_img('v_data/test/cars/5.jpg', target_size=(227, 227))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,227,227,3)
label = model_ann.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", np.argmax(label,axis=1))

In [ ]:
type(test_data['image'])

pandas.core.series.Series